## In this notebook i will create and train the first prediction model - Random Forest

In [3]:
import pandas as pd

df = pd.read_csv(
    r"C:\Users\ADMIN\Desktop\Coding_projects\stock_market_prediction\Stock-Market-Prediction\data\processed\gemini_with_trends.csv",
    index_col=0
)


In [4]:
print(df)


              high       low     close  Volume BTC    roc_4h   roc_24h  \
open                                                                     
245.00      245.00    244.50    245.00    4.453649       NaN       NaN   
245.00      245.00    244.92    244.92    3.016926       NaN       NaN   
244.92      244.92    244.25    244.25    3.895252       NaN       NaN   
244.25      244.99    244.02    244.99    3.920632       NaN       NaN   
244.99      244.99    244.00    244.00    3.690472 -0.004082       NaN   
...            ...       ...       ...         ...       ...       ...   
87192.76  87433.40  87000.00  87063.30   40.926363  0.001191  0.005838   
87063.30  87335.11  86930.18  87310.36   18.377515  0.005740  0.000333   
87310.36  87710.97  87226.71  87551.64   18.607755  0.007583  0.004471   
87551.64  87603.05  87298.29  87346.45   10.438136  0.001763  0.005893   
87346.45  87346.45  87101.82  87223.20   12.074964  0.001837  0.003635   

          roc_7days  roc_30days  
ope

In [ ]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.metrics import mean_absolute_error

from eli5.sklearn import PermutationImportance
import eli5

ImportError: cannot import name '_approximate_mode' from 'sklearn.utils.extmath' (c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\extmath.py)

In [20]:
pip install eli5

  Using cached scikit_learn-1.6.1-cp310-cp310-win_amd64.whl.metadata (15 kB)
Using cached scikit_learn-1.6.1-cp310-cp310-win_amd64.whl (11.1 MB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.3.2
    Uninstalling scikit-learn-1.3.2:
      Successfully uninstalled scikit-learn-1.3.2
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [21]:
df["target"] = df["close"].pct_change().shift(-1)
df = df.dropna(subset=["target"])

feature_cols = [c for c in df.columns if c not in ("timestamp", "target")]
X_all = df[feature_cols]
y = df["target"]

In [22]:
tscv = TimeSeriesSplit(n_splits=5, gap=0)

rf = RandomForestRegressor(
    n_estimators=500,
    max_depth=None,
    n_jobs=-1,
    random_state=42
)

cv_scores = cross_val_score(
    rf, X_all, y,
    cv=tscv,
    scoring="neg_mean_absolute_error"
)

print("MAE per split:", -cv_scores.round(6))
print("Average MAE  :", -cv_scores.mean())

MAE per split: [0.011255 0.004664 0.010861 0.003847 0.003866]
Average MAE  : 0.0068985085233682995


In [27]:
pip install scikit-learn==1.2.2


   ---------------------------------------- 0.0/8.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.3 MB 1.3 MB/s eta 0:00:07
    --------------------------------------- 0.1/8.3 MB 1.3 MB/s eta 0:00:07
   - -------------------------------------- 0.2/8.3 MB 1.7 MB/s eta 0:00:05
   -- ------------------------------------- 0.4/8.3 MB 2.5 MB/s eta 0:00:04
   --- ------------------------------------ 0.6/8.3 MB 2.9 MB/s eta 0:00:03
   ---- ----------------------------------- 0.8/8.3 MB 3.2 MB/s eta 0:00:03
   ------ --------------------------------- 1.3/8.3 MB 4.1 MB/s eta 0:00:02
   ----------- ---------------------------- 2.4/8.3 MB 6.6 MB/s eta 0:00:01
   --------------------- ------------------ 4.5/8.3 MB 11.0 MB/s eta 0:00:01
   ---------------------------------- ----- 7.1/8.3 MB 15.6 MB/s eta 0:00:01
   ---------------------------------------- 8.3/8.3 MB 17.1 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
   

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 1.4.6 requires scikit-learn>=1.3.2, but you have scikit-learn 1.2.2 which is incompatible.
eli5 0.15.0 requires scikit-learn>=1.6.0, but you have scikit-learn 1.2.2 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [30]:
pip uninstall scikit-learn eli5 -y


Found existing installation: scikit-learn 1.2.2
Uninstalling scikit-learn-1.2.2:
  Successfully uninstalled scikit-learn-1.2.2
Found existing installation: eli5 0.15.0
Uninstalling eli5-0.15.0:
  Successfully uninstalled eli5-0.15.0
Note: you may need to restart the kernel to use updated packages.


You can safely remove it manually.


In [31]:
pip install scikit-learn==1.2.2 eli5==0.13.0


  Using cached scikit_learn-1.2.2-cp310-cp310-win_amd64.whl.metadata (11 kB)
     ---------------------------------------- 0.0/216.2 kB ? eta -:--:--
     ----- --------------------------------- 30.7/216.2 kB 1.3 MB/s eta 0:00:01
     ------------------- ------------------ 112.6/216.2 kB 1.1 MB/s eta 0:00:01
     -------------------------------------- 216.2/216.2 kB 1.5 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Using cached scikit_learn-1.2.2-cp310-cp310-win_amd64.whl (8.3 MB)
  Created wheel for eli5: filename=eli5-0.13.0-py2.py3-none-any.whl size=107853 sha256=fc2a527466bd757e677d1d7380cd77f6296c7d3d93474a9b0f45ba4848b3ab12
  Stored in directory: c:\users\admin\appdata\local\pip\cache

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 1.4.6 requires scikit-learn>=1.3.2, but you have scikit-learn 1.2.2 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [32]:
from eli5.sklearn import PermutationImportance
import eli5

import eli5, importlib, pkg_resources
print("eli5 version:", pkg_resources.get_distribution("eli5").version)

TypeError: C function sklearn.utils._vector_sentinel.__pyx_fuse_0vector_to_nd_array has wrong signature (expected PyArrayObject *(std::vector<__pyx_t_7sklearn_5utils_9_typedefs_DTYPE_t>  *), got PyArrayObject *(std::vector<__pyx_t_7sklearn_5utils_9_typedefs_float64_t>  *))

In [29]:
# %% [code]
rf.fit(X_all, y)

perm = PermutationImportance(
    rf,
    scoring="neg_mean_absolute_error",
    random_state=42,
    n_iter=5,
    n_jobs=-1
)
perm.fit(X_all, y)

# Nicely formatted HTML output in Jupyter
eli5.show_weights(
    perm,
    feature_names=feature_cols,
    top=len(feature_cols)   # show all features
)


KeyboardInterrupt: 